In [1]:
import math
import numpy as np

In [2]:
train = np.loadtxt('DS2train.csv',delimiter = ',')
val = np.loadtxt('DS2val.csv',delimiter = ',')
test = np.loadtxt('DS2test.csv',delimiter = ',')
print("len(test)", len(test))

len(test) 800


#GDA

In [3]:
class0_count = 0
class1_count = 0
u1 = 0
u0 = 0
x = 0
s0 = 0
s1 = 0

In [4]:
#Mean and Probs

len_train = len(train)
for row in train:
    if row[-1] == 0:
        class0_count += 1
        u0 += row[:-1]

for row in train:
    if row[-1] == 1:
        class1_count += 1
        u1 += row[:-1]

Prob_0 = class0_count/len_train
Prob_1 = class1_count/len_train

u0 /= class0_count
u1 /= class1_count

In [5]:
#Covariance matrix

for row in train:
    if row[-1]==0:
        x = np.array(row[:-1])
        x -= u0
        x = np.reshape(x,(20,1))       
        s0 += x.dot(x.T)

s0  /= class0_count


for row in train:
    if row[-1]==1:
        x = np.array(row[:-1])
        x -= u1
        x = np.reshape(x,(20,1))    
       
        s1 += x.dot(x.T)

s1 /= class1_count

    
cov_mat = Prob_0*s0+Prob_1*s1

In [6]:
print ("Prob_0", Prob_0)
print ("Prob_1", Prob_1)
print ("mean for class 0", u0)
print ("mean for class 1", u1)
print ("cov_mat", cov_mat)
print ("cov_mat.shape", cov_mat.shape)

Prob_0 0.5
Prob_1 0.5
mean for class 0 [1.2041308  1.17971328 1.12557598 1.13664765 1.19837987 1.1738258
 1.16164335 1.18582777 1.19632186 1.1697364  1.16279209 1.2539347
 1.17506621 1.14821744 1.18675745 1.18718042 1.17210468 1.20681792
 1.17941305 1.14182175]
mean for class 1 [0.99822882 1.00742219 0.95234028 0.9538514  1.01124007 1.04625583
 0.89283352 1.00489783 0.950542   0.93972481 0.95023072 0.9723977
 0.94185275 0.93768056 0.96042636 0.95321317 1.00062109 1.02011701
 0.93567492 0.9725729 ]
cov_mat [[7.90877842 5.4700443  4.651385   5.04634762 4.45064495 5.80515233
  5.92663151 5.61982931 4.76280329 5.33260806 5.7014855  5.15769679
  5.10698751 6.25721194 5.44648303 5.77935318 5.53322689 5.47641766
  5.69956115 6.0482854 ]
 [5.4700443  7.41773284 4.94931792 5.22484304 5.07962879 6.08559771
  6.55506079 5.39323    4.69668615 5.16963768 4.98578811 5.10111108
  4.97617224 6.20585889 5.71791783 5.98207124 5.42023995 5.08174077
  6.25619273 5.84191833]
 [4.651385   4.94931792 6.80899

In [7]:
#w and w0

cov_mat_inv = np.linalg.inv(cov_mat)
w = cov_mat_inv.dot(u0-u1)
w0 = (-.5*(u0.T).dot(cov_mat_inv).dot(u0))+(.5*(u1.T).dot(cov_mat_inv).dot(u1))+(np.log(Prob_0/Prob_1))
print("w",w)
print("w0",w0)

w [-0.00966259 -0.02559834 -0.07705795 -0.00013703  0.03761088 -0.11042393
  0.00399542 -0.01742475  0.09723131  0.02207119 -0.00945184  0.05906434
  0.04334472  0.03385311 -0.01117373  0.04825551 -0.04702542 -0.0178343
  0.02450182  0.01071567]
w0 -0.055699759777591396


In [8]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x)) 

def prob_class0(x):
    a = w.dot(x)+w0   
    return sigmoid(a) 

def prob_class1(x):
    return 1-prob_class0(x)

In [9]:
# test accuracy, precision, recall, and F-measure
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0
len_test = len(test)

for row in test:
    x = np.array(row[:-1])
    true_value = row[-1]
    prob0 = prob_class0(x)

    choice = 0
    if prob0<.5:
        choice=1
        
        
    if choice == 0:
        if true_value == 0: 
            true_neg += 1 #true -ve
        else: 
            false_neg += 1 #false -ve
    else:
        if true_value == 0: 
            false_pos += 1 #false +ve
        else: 
            true_pos += 1 #true +ve
    
accuracy = (true_neg+true_pos)/len_test
precision = true_pos/(true_pos+false_pos) 
recall = true_pos/(true_pos+false_neg)
F_measure = 2*precision*recall/(precision+recall)

In [10]:
print("Accuracy =", accuracy)
print("Precision", precision)
print("Recall =", recall)
print("F Measure =", F_measure)

Accuracy = 0.49875
Precision 0.4986737400530504
Recall = 0.47
F Measure = 0.4839124839124839


#KNN

In [11]:
def classifier(x,k_value): 
    neighbour = []
    avg_val = 0
    for i,row in enumerate(train):
        point = np.array(row[:-1])
        dif = x-point
        euc_distance = np.dot(dif,dif)  
        if len(neighbour)<k_value:
            neighbour.append([euc_distance,i])
        else:
            neighbour.sort(key=lambda x: x[0])
           
            if euc_distance<neighbour[k_value-1][0]:
                del neighbour[-1]
                neighbour.append([euc_distance,i]) 
                
    for n in neighbour:
        i = n[1] 
        val = train[i][-1]
        avg_val+=val
    
    avg_val/=k_value
    

    if avg_val<.5:
        return 0
    else:
        return 1

In [12]:
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0
len_test = len(test)
for k_value in range(1,6):
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    for row in test:
        x = np.array(row[:-1])
        true_val = row[-1]
        choice = classifier(x,k_value)
        if choice == 0:
            if true_val == 0: 
                true_neg+=1
            else: 
                false_neg+=1
        else:
            if true_val == 0: 
                false_pos+=1
            else: 
                true_pos+=1

    accuracy = (true_neg+true_pos)/len_test
    precision = true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos+false_neg)
    F_measure = 2*precision*recall/(precision+recall)
    
    
    print('\n')
    print("for k value =", k_value)
    print("Accuracy =", accuracy)
    print("Precision =", precision)
    print("Recall =", recall)
    print("F Measure =", F_measure)



for k value = 1
Accuracy = 0.5275
Precision = 0.5276381909547738
Recall = 0.525
F Measure = 0.5263157894736842


for k value = 2
Accuracy = 0.47625
Precision = 0.483704974271012
Recall = 0.705
F Measure = 0.5737538148524923


for k value = 3
Accuracy = 0.53125
Precision = 0.5318066157760815
Recall = 0.5225
F Measure = 0.5271122320302648


for k value = 4
Accuracy = 0.52
Precision = 0.5147058823529411
Recall = 0.7
F Measure = 0.5932203389830508


for k value = 5
Accuracy = 0.525
Precision = 0.5247524752475248
Recall = 0.53
F Measure = 0.527363184079602
